In [1]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.2 MB/s eta 0:00:00


In [11]:
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from google.colab import drive
import os
from os import listdir
from transformers import AutoFeatureExtractor, ResNetForImageClassification
import torch
import torchvision
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [19]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Define a transformation for input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess an example image
image_path = 'path/to/your/image.jpg'
image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)

# Make a prediction
with torch.no_grad():
    output = model(input_batch)

# Get the predicted class
_, predicted_class = output.max(1)

# Print the predicted class label
print(f"Predicted class label: {predicted_class.item()}")


torch.Size([3, 275, 183])
torch.Size([3, 307, 460])
torch.Size([3, 731, 1000])
torch.Size([3, 675, 1200])
torch.Size([3, 168, 299])
torch.Size([3, 600, 1000])
torch.Size([4, 670, 670])
torch.Size([3, 776, 1200])
torch.Size([3, 787, 1400])
torch.Size([3, 183, 275])
torch.Size([3, 1000, 1500])
torch.Size([3, 1400, 1400])
torch.Size([3, 1488, 2048])
torch.Size([3, 175, 289])
torch.Size([3, 408, 612])
torch.Size([3, 275, 183])
torch.Size([3, 1089, 800])
torch.Size([3, 808, 1200])
torch.Size([3, 800, 1200])
torch.Size([3, 900, 1200])
torch.Size([3, 572, 1040])
torch.Size([3, 1707, 2560])
torch.Size([3, 1200, 1200])
torch.Size([3, 183, 275])
torch.Size([3, 1400, 1400])
torch.Size([3, 420, 630])
torch.Size([3, 1024, 819])
torch.Size([3, 1050, 1400])
torch.Size([3, 1440, 2560])
torch.Size([3, 1707, 2560])
torch.Size([3, 348, 620])
torch.Size([3, 1798, 2697])
torch.Size([3, 1200, 1200])
torch.Size([3, 800, 1200])
torch.Size([3, 450, 600])
torch.Size([3, 1200, 1200])
torch.Size([3, 675, 1200])
t

In [20]:
import os
import torch
import torchvision
from transformers import AutoFeatureExtractor, ResNetForImageClassification

# Load the feature extractor and model outside the loop
model_name = "microsoft/resnet-18"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = ResNetForImageClassification.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Lists to store uncertain examples
uncertain_images = []
uncertain_confidences = []
folder_dir = '/content/drive/My Drive/Ai TP/dataset_sport/'

for picture in os.listdir(folder_dir):
    image_path = os.path.join(folder_dir, picture)
    image = torchvision.io.read_image(image_path)
    print(image.shape)

    # Preprocess the image
    inputs = torchvision.transforms.functional.normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    inputs = inputs.unsqueeze(0)  # Add a batch dimension

    with torch.no_grad():
        logits = model(inputs).logits

    # Apply softmax to get confidence scores
    softmax_probs = torch.nn.functional.softmax(logits[0], dim=0)

    # Model predicts one of the 1000 ImageNet classes
    predicted_label = logits.argmax(-1).item()
    confidence = softmax_probs[predicted_label].item() * 100

    # Print prediction details
    print(f"Image: {image_path}")
    print(f"Predicted label: {model.config.id2label[predicted_label]}")
    print(f"Confidence: {confidence:.2f}%")

    # Check if confidence is below a certain threshold (adjust as needed)
    uncertainty_threshold = 70.0
    if confidence < uncertainty_threshold:
        uncertain_images.append(image_path)
        uncertain_confidences.append(confidence)

# Print uncertain examples
print("\nUncertain Examples:")
for i, image_path in enumerate(uncertain_images):
    print(f"Image {i + 1}: {image_path}")
    print(f"Confidence: {uncertain_confidences[i]:.2f}%\n")


torch.Size([3, 275, 183])


TypeError: ignored